# LSTM & Redes Neuronales
En esta sección se realizara buscara darle una solucion al problema por medio de un combinacion de una Long Short Term Memory (LSTM) y una Red Neuronal

In [1]:
import pandas as pd

# Cargo los datasets

In [ ]:
X_

,fecha,altura_max_m,ganancia_altura_m,duracion_min,distancia_km,velocidad_promedio_kmh,lat_despegue,lon_despegue,hora_despegue,num_termicas,...,precipitation_avg,temp_2m_avg,cloud_cover_avg,wind_u_avg,wind_v_avg,pressure_avg,boundary_layer_height_avg,cape_avg,skin_temp_avg,wind_speed_avg
310,2025-01-30,2179.0,2130.0,353.04,817.8,121.4,-34.475717,-59.420967,15:05:00,184.0,...,0.0,29.055103,0.057043,-2.356329,-1.069902,994.892500,978.727867,87.7875,32.395239,2.736307
59,2024-10-11,2496.0,2393.0,221.70,477.8,129.3,-31.276550,-61.508000,15:32:00,26.0,...,0.0,22.988965,0.000000,-0.285249,-1.285338,991.744250,598.029073,0.0000,27.065283,1.388841
163,2025-01-05,1834.0,1626.0,196.50,431.2,131.7,-38.036233,-60.135417,17:17:00,53.0,...,0.0,32.966699,0.145804,-3.621085,-2.994528,991.147125,1578.745844,0.0000,38.794507,4.836791
54,2024-09-28,1927.0,1814.0,219.60,390.7,106.7,-31.286383,-61.512383,16:13:00,138.0,...,0.0,24.374878,0.000000,-3.055090,-3.341286,994.104187,875.217896,0.0000,29.200513,4.678033
267,2025-01-12,2279.0,2074.0,283.00,643.1,136.4,-38.032867,-60.130783,16:20:00,86.0,...,0.0,30.779639,0.071082,-3.270447,-1.496068,994.458625,1343.225156,0.0000,35.753687,3.763343
